In [ ]:
from __future__ import unicode_literals

import sys, os
BIN = os.path.expanduser("../../../")
sys.path.append(BIN)

import numpy as np
from scipy.constants import m_p, c, e, pi
import matplotlib.pyplot as plt
%matplotlib inline

import copy
import itertools

from PyHEADTAIL_test_tools import generate_objects, BunchTracker, track, compare_traces, compare_projections, Machine

from PyHEADTAIL.feedback.feedback import OneboxFeedback, Kicker, PickUp
from PyHEADTAIL.feedback.processors.multiplication import ChargeWeighter
from PyHEADTAIL.feedback.processors.linear_transform import Averager
from PyHEADTAIL.feedback.processors.misc import Bypass
from PyHEADTAIL.feedback.processors.register import Register, TurnDelay

np.random.seed(0)

In [ ]:
""" 
    This test demonstrates multi bunch simulations with a transverse feedback system. The test is identical
    to the test 002, expect multiple bunches have been generated by giving a filling scheme to the object
    generators and a flag 'mpi = True' to the OneboxFeedback, Pickup and Kicker objects.
    

"""

In [ ]:
# Basic parameters for the simulation
n_macroparticles = 1000
n_slices = 10
n_segments = 5
n_sigma_z = 4

n_turns = 50

# Longitudinal motion of the bunch is not taken into account in this example.
machine = Machine(n_segments= n_segments)
# The longitudinal motion can be included to simulations by uncommenting the following line
# machine = Machine(Q_s = 0.0020443)
first_index = 10 #[buckets]
batch_spacing = 100  #[buckets]
n_batches = 2
n_bunches_per_batch = 20
bunch_spacing = 10 #[buckets]

batch_separation = batch_spacing+n_bunches_per_batch* bunch_spacing

filling_scheme = []
for j in xrange(n_batches):
    for i in xrange(n_bunches_per_batch):
        filling_scheme.append(first_index + i * bunch_spacing + j*batch_separation)


print filling_scheme
bunch_ref, slicer_ref,trans_map, long_map = generate_objects(machine, n_macroparticles, 
                                                             n_slices,n_sigma_z,
                                                             filling_scheme=filling_scheme)


In [ ]:
oscillation_lambda = 50. # [m]
oscillation_amplitide = 1e-3 # [m]

bunch_ref.x += oscillation_amplitide*np.sin(2*np.pi*bunch_ref.z/oscillation_lambda)
bunch_ref.y += oscillation_amplitide*np.sin(2*np.pi*bunch_ref.z/oscillation_lambda)
    
bunch_init = copy.deepcopy(bunch_ref)
tracker_ref = BunchTracker(bunch_init)
maps_ref = [i for i in trans_map]
# + [long_map]
track(n_turns, bunch_init,maps_ref,tracker_ref)

In [ ]:
feedback_gain = 0.1
# feedback_gain = (0.1,0.4)

# Parameters for the registers
delay = 1 
n_values = 3

In [ ]:
# A reference feedback system, which is originally introduced in the file 001_ideal_feedbacks.ipynb:

bunch_OneBox_bunch = copy.deepcopy(bunch_ref)
tracker_OneBox_bunch = BunchTracker(bunch_OneBox_bunch)
slicer_OneBox_bunch = copy.deepcopy(slicer_ref)

processors_bunch_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]
processors_bunch_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager()
]

feedback_map = OneboxFeedback(feedback_gain,slicer_OneBox_bunch,processors_bunch_x,processors_bunch_y)
total_map_OneBox_bunch = [i for i in trans_map] + [feedback_map]
#  + [long_map]

track(n_turns, bunch_OneBox_bunch,total_map_OneBox_bunch,tracker_OneBox_bunch)

In [ ]:
bunch_register_example = copy.deepcopy(bunch_ref)
tracker_register_example = BunchTracker(bunch_register_example)
slicer_register_example = copy.deepcopy(slicer_ref)

processors_register_example_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    TurnDelay(delay, machine.Q_x, n_values)
]
processors_register_example_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    TurnDelay(delay, machine.Q_y, n_values)
]

feedback_map = OneboxFeedback(feedback_gain,slicer_register_example,
                              processors_register_example_x,processors_register_example_y, mpi = True)
total_map_register_example = [feedback_map] + [i for i in trans_map]
#  + [long_map]

track(n_turns, bunch_register_example,total_map_register_example,tracker_register_example)


In [ ]:
bunch_separated_example = copy.deepcopy(bunch_ref)
tracker_separated_example = BunchTracker(bunch_separated_example)
slicer_separated_example = copy.deepcopy(slicer_ref) 

pickup_beta_x = machine.beta_x
pickup_beta_y = machine.beta_y

kicker_beta_x = machine.beta_x
kicker_beta_y = machine.beta_y

pickup_location_x = 1.*2.*pi/float(n_segments)*machine.Q_x
pickup_location_y = 1.*2.*pi/float(n_segments)*machine.Q_y

kicker_location_x = 2.*2.*pi/float(n_segments)*machine.Q_x
kicker_location_y = 2.*2.*pi/float(n_segments)*machine.Q_y

# Exactly same signal processors as in the previous example are used in the pickup
processors_pickup_x = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    Register(n_values, machine.Q_x, delay)
]
processors_pickup_y = [
    ChargeWeighter(normalization = 'segment_average'),
    Averager(),
    Register(n_values, machine.Q_y, delay)
]

# The map element is created by giving a slicer object, the signal processors and the beam parameters as input parameters
pickup_map = PickUp(slicer_separated_example,processors_pickup_x,processors_pickup_y, pickup_location_x, pickup_beta_x,
                 pickup_location_y, pickup_beta_y, mpi=True)


# In this example, the signals are not modified in the kicker, i.e. only bypass processors are used
processors_kicker_x = [Bypass()]
processors_kicker_y = [Bypass()]

# A list of references to registers from the signal processors(s) of the pickup(s)
registers_x = [processors_pickup_x[-1]]
registers_y = [processors_pickup_y[-1]]

kicker_map = Kicker(feedback_gain, slicer_separated_example, processors_kicker_x, processors_kicker_y,
                    registers_x, registers_y, kicker_location_x, kicker_beta_x, kicker_location_y, kicker_beta_y,
                    mpi=True)

# Finally the maps must be added to correct slots of the total map determined by the betaton phase advance locations of 
# the picup and the kicker
total_map_separated_example = [trans_map[0]] + [pickup_map] + [trans_map[1]] + [kicker_map]
for element in trans_map[2:]:
    total_map_separated_example += [element]
# total_map_separated_example += [long_map]
    
    
track(n_turns, bunch_separated_example,total_map_separated_example,tracker_separated_example)

In [ ]:
# In here, the traces and the projections from different implementations of the feedback system are compared.
# Note: the properties are calculated over all particles in all bunches
# Note: the scale in the emittance figures

compare_traces([tracker_OneBox_bunch,tracker_register_example,tracker_separated_example],
               ['Ideal', 'Delayed', 'Separated'])
compare_projections([ bunch_OneBox_bunch,  bunch_register_example, bunch_separated_example], 
                    ['Ideal', 'Delayed', 'Separated'], n_particles = 1000)

# compare_traces([tracker_OneBox_bunch,tracker_register_example],
#                ['Ideal', 'Delayed', 'Separated'])
# compare_projections([ bunch_OneBox_bunch,  bunch_register_example], 
#                     ['Ideal', 'Delayed', 'Separated'])

Jani Komppula, CERN, 2017